In [1]:
import torch
import torch.optim
import numpy as np

from nni.nas.space import model_context
from nni.nas.strategy import DARTS as DartsStrategy
from nni.nas.experiment import NasExperiment
from dataset.regression import fetch_data
from evaluators.regression import RegressionEvaluator
from models.mlp import MLP

In [2]:
np.random.seed(0)
torch.random.manual_seed(0)

# Fetch dataset loaders

In [3]:
task_config, loaders = fetch_data(batch_size=256, num_workers=4)

In [4]:
for split_name, loader in loaders.items():
    print(split_name, 'dataset size:', len(loader.dataset))

train dataset size: 13209
val dataset size: 3303
test dataset size: 4128


# Training

In [5]:
evaluator = RegressionEvaluator(
    learning_rate=3e-4,
    weight_decay=1e-5,
    optimizer=torch.optim.AdamW,
    train_dataloaders=loaders['train'],
    val_dataloaders=loaders['val'],
    max_epochs=100,
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [6]:
strategy = DartsStrategy()
model_space = MLP(d_in=task_config.in_features, d_out=task_config.out_features, dropout=0.1)
experiment = NasExperiment(model_space, evaluator, strategy)
experiment.run()

[2024-05-14 01:46:00] Config is not provided. Will try to infer.
[2024-05-14 01:46:00] Strategy is found to be a one-shot strategy. Setting execution engine to "sequential" and format to "raw".
[2024-05-14 01:46:00] WARNING: `training_service` will be ignored for sequential execution engine.
[2024-05-14 01:46:00] WARNING: `training_service` will be ignored for sequential execution engine.
[2024-05-14 01:46:00] WARNING: `training_service` will be ignored for sequential execution engine.
[2024-05-14 01:46:00] WARNING: `training_service` will be ignored for sequential execution engine.
[2024-05-14 01:46:00] WARNING: `training_service` will be ignored for sequential execution engine.
[2024-05-14 01:46:00] WARNING: `training_service` will be ignored for sequential execution engine.
[2024-05-14 01:46:00] WARNING: `training_service` will be ignored for sequential execution engine.
[2024-05-14 01:46:01] WARNING: `training_service` will be ignored for sequential execution engine.
[2024-05-14 01

You are using a CUDA device ('NVIDIA GeForce RTX 4060 Ti') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name            | Type             | Params
-----------------------------------------------------
0 | training_module | RegressionModule | 2.6 M 
-----------------------------------------------------
2.6 M     Trainable params
0         Non-trainable params
2.6 M     Total params
10.527    Total estimated model params size (MB)


Training: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=100` reached.


[2024-05-14 01:50:10] Waiting for models submitted to engine to finish...
[2024-05-14 01:50:10] Experiment is completed.
[2024-05-14 01:50:10] WARNING: `training_service` will be ignored for sequential execution engine.


/home/sisha/miniconda3/envs/test_env/lib/python3.12/site-packages/nni/nas/evaluator/pytorch/lightning.py:311: Multiple metrics without "default" is not supported by current framework.


True

# Train final model

In [7]:
exported_arch = experiment.export_top_models(formatter='dict')[0]
print(exported_arch)

{'MLP/d_block': 32, 'MLP/in_act': 1, 'MLP/n_blocks': 2, 'MLP/blocks_act': 0}


In [8]:
with model_context(exported_arch):
    final_model = MLP(d_in=task_config.in_features, d_out=task_config.out_features, dropout=0.1)

In [9]:
evaluator = RegressionEvaluator(
    learning_rate=3e-4,
    weight_decay=1e-5,
    optimizer=torch.optim.AdamW,
    train_dataloaders=loaders['train'],
    val_dataloaders=loaders['val'],
    max_epochs=150,
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [10]:
evaluator.fit(final_model)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type       | Params
-----------------------------------------
0 | criterion | MSELoss    | 0     
1 | metrics   | ModuleDict | 0     
2 | _model    | MLP        | 2.4 K 
-----------------------------------------
2.4 K     Trainable params
0         Non-trainable params
2.4 K     Total params
0.010     Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

[2024-05-14 01:51:04] Intermediate result: {'mse': 0.9081547856330872, 'rmse': 0.951565682888031, 'default': 0.9081547856330872}  (Index 0)


...

[2024-05-14 01:52:16] Intermediate result: {'mse': 0.21641142666339874, 'rmse': 0.4626224935054779, 'default': 0.21641142666339874}  (Index 148)


Validation: |          | 0/? [00:00<?, ?it/s]

[2024-05-14 01:52:17] Intermediate result: {'mse': 0.21811595559120178, 'rmse': 0.4643125534057617, 'default': 0.21811595559120178}  (Index 149)


`Trainer.fit` stopped: `max_epochs=150` reached.


[2024-05-14 01:52:17] Final result: {'mse': 0.21811595559120178, 'rmse': 0.4643125534057617, 'default': 0.21811595559120178}


# Evaluate final model

In [11]:
test_evaluator = RegressionEvaluator(
    learning_rate=3e-4,
    weight_decay=1e-5,
    optimizer=torch.optim.AdamW,
    val_dataloaders=loaders['test'],
    max_epochs=100,
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [12]:
test_evaluator.evaluate(final_model)

[2024-05-14 01:52:59] Only validation dataloaders are available. Skip to validation.


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Validation: |          | 0/? [00:00<?, ?it/s]

[2024-05-14 01:52:59] Intermediate result: {'mse': 0.23070895671844482, 'rmse': 0.4781631529331207, 'default': 0.23070895671844482}  (Index 150)
[2024-05-14 01:52:59] Final result: {'mse': 0.23070895671844482, 'rmse': 0.4781631529331207, 'default': 0.23070895671844482}
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
     Validate metric           DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       val_default          0.23070895671844482
        val_loss            0.23070895671844482
         val_mse            0.23070895671844482
        val_rmse            0.4781631529331207
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'val_loss': 0.23070895671844482,
  'val_mse': 0.23070895671844482,
  'val_rmse': 0.4781631529331207,
  'val_default': 0.23070895671844482}]